In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing
from torch import nn
from tensordict.nn import TensorDictModule

from collections import defaultdict

In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing

In [2]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

In [3]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

channel = EngineConfigurationChannel()
env_path = "C:/Users/Pawel/Documents/Unity_Project/warehouse-bot-training/environment_builds/warehouse_step1_full/Warehouse_Bot.exe"

from torchrl.envs import UnityMLAgentsEnv

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  # additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=3)

c:\Users\Pawel\anaconda3\envs\mlagents\lib\site-packages\mlagents_envs\environment.py:94: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  unity_communicator_version = StrictVersion(unity_com_ver)


#### Transform environment from `mlagents` to `gymnasium`

In [4]:
import gymnasium as gym

In [5]:
print(gym.__version__)

1.0.0


In [6]:
from env_gymnasium_wrapper import UnityGymWrapper 

gymnasium_env = UnityGymWrapper(unity_env)

#### Creating stable_baselines3 model

In [7]:
from stable_baselines3 import PPO

model = PPO.load("./saved_models/warehouse_step1_full_1", gymnasium_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
env = model.get_env()
obs = env.reset()
for i in range(500):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)